In [1]:
import json 
import numpy as np 
import whisper
import os.path as osp
import os, glob

## 데이터셋 처리 

> __data_process 함수__ 
>
> aihub의 [소음 환경 음성인식 데이터](https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&dataSetSn=568)를 처리하는 함수로, 다른 데이터는 텍스트 처리가 달라질 수 있습니다.
>
> 경로를 입력으로 받아 웨이브폼과 텍스트로 각각 처리합니다.
> 
> 발화자가 바뀔 때를 기준으로 오디오를 잘라 리스트 형태로 반환합니다. (audio_arr, text_arr)

> 텍스트 전처리(preprocess 함수)의 경우 https://github.com/sooftware/ksponspeech 를 참조하였습니다. 위스퍼도 숫자를 포함한 추론 결과를 내기 때문에 아래 코드에서는 option2를 선택하였습니다. 
>   
> - Option1 : phonetic transcript     
> ``칠 십 퍼센트 확률이라니 아 모 몬 소리야 진짜 백 프로가 왜 안돼?``       
> - Option2 : spelling transcript     
> ``70% 확률이라니 아 뭐 뭔 소리야 진짜 100%가 왜 안돼?``       

In [6]:
#이 코드 블록은 ksponspeech/preprocess/preprocess.py에서 복사해온 코드입니다. 
import os
import re
from joblib import Parallel, delayed, cpu_count
from tqdm import tqdm


def bracket_filter(sentence, mode='phonetic'):
    new_sentence = str()

    if mode == 'phonetic':
        flag = False

        for ch in sentence:
            if ch == '(' and flag is False:
                flag = True
                continue
            if ch == '(' and flag is True:
                flag = False
                continue
            if ch != ')' and flag is False:
                new_sentence += ch

    elif mode == 'spelling':
        flag = True

        for ch in sentence:
            if ch == '(':
                continue
            if ch == ')':
                if flag is True:
                    flag = False
                    continue
                else:
                    flag = True
                    continue
            if ch != ')' and flag is True:
                new_sentence += ch

    else:
        raise ValueError("Unsupported mode : {0}".format(mode))

    return new_sentence


def special_filter(sentence, mode='phonetic', replace=None):
    SENTENCE_MARK = ['?', '!', '.']
    NOISE = ['o', 'n', 'u', 'b', 'l']
    EXCEPT = ['/', '+', '*', '-', '@', '$', '^', '&', '[', ']', '=', ':', ';', ',']

    new_sentence = str()
    for idx, ch in enumerate(sentence):
        if ch not in SENTENCE_MARK:
            if idx + 1 < len(sentence) and ch in NOISE and sentence[idx + 1] == '/':
                continue

        if ch == '#':
            new_sentence += '샾'

        elif ch == '%':
            if mode == 'phonetic':
                new_sentence += replace
            elif mode == 'spelling':
                new_sentence += '%'

        elif ch not in EXCEPT:
            new_sentence += ch

    pattern = re.compile(r'\s\s+')
    new_sentence = re.sub(pattern, ' ', new_sentence.strip())
    return new_sentence


def sentence_filter(raw_sentence, mode, replace=None):
    return special_filter(bracket_filter(raw_sentence, mode), mode, replace)


PERCENT_FILES = {
    '087797': '퍼센트',
    '215401': '퍼센트',
    '284574': '퍼센트',
    '397184': '퍼센트',
    '501006': '프로',
    '502173': '프로',
    '542363': '프로',
    '581483': '퍼센트'
}


def read_preprocess_text_file(file_path, mode):
    with open(file_path, 'r', encoding='cp949') as f:
        raw_sentence = f.read()
        file_name = os.path.basename(file_path)
        if file_name[12:18] in PERCENT_FILES.keys():
            replace = PERCENT_FILES[file_name[12:18]]
        else:
            replace = None
        return sentence_filter(raw_sentence, mode=mode, replace=replace)


def preprocess(dataset_path, mode='phonetic'):
    print('preprocess started..')

    audio_paths = []
    transcripts = []

    with Parallel(n_jobs=cpu_count() - 1) as parallel:

        for idx, subfolder in tqdm(list(enumerate(os.listdir(dataset_path))), desc=f'Preprocess text files on {dataset_path}'):
            path = os.path.join(dataset_path, subfolder)

            # list-up files
            sub_file_list = [
                os.path.join(path, file_name) for file_name in os.listdir(path) if file_name.endswith('.json')
            ]
            audio_sub_file_list = [
                os.path.join(subfolder, file_name)
                for file_name in os.listdir(path) if file_name.endswith('.json')
            ]

            # do parallel and get results
            new_sentences = parallel(
                delayed(read_preprocess_text_file)(p, mode) for p in sub_file_list
            )

            audio_paths.extend(audio_sub_file_list)
            transcripts.extend(new_sentences)

    return audio_paths, transcripts

> __오디오 파일과 텍스트 파일을 처리해줍니다.__
> 오디오 파일을 벡터로 변환시키는 이유는 허깅페이스에 업로드하기 위함입니다. 허깅페이스 업로드를 하지 않을 시 text만 전처리하도록 수정을 권장드립니다. 

In [11]:
SAMPLE_RATE =16000  

def data_process(wav: str, annotation:str,): 
    """
    loading paths into waveform and text.
    Args
        wav(str) : path of .wav or .mp3 file 
        annotation(str) : patho of annotation 
    Return
        audio_arr (list) : list of numpy array containing wave form sampled in 16kHz 
        text_arr (list) : list of sentences.
    """
    audio = whisper.load_audio(wav) #오디오파일을 벡터 값으로 불러옴 - sr: 16kHz

    with open(annotation) as f: #Json 파일 가져옴 
        data = json.load(f)       

    audio_arr, text_arr = [], [] 
    for row in data['dialogs']: 
        start = int(row['startTime']) * SAMPLE_RATE  #발화자의 발화 시작 시간 
        end = int(row['endTime']) * SAMPLE_RATE #발화 종료 시간 
        audio_arr.append(audio[start:end]) #발화자의 시작-종료 시간만 가져옴 

        sentence = sentence_filter(row['speakerText'], mode='spelling')
        text_arr.append(sentence)
    return audio_arr, text_arr

In [35]:
data_dir = "/home/yerang/문서/ASR/python_speech_recognition/aihub/06.지하철,버스/02.지하철안"
wav_fns = glob.glob(osp.join(data_dir, '**', '*.wav'), recursive=True)
print(f" number of files: {len(wav_fns)}") 

audios = [] 
texts = []
for wav in wav_fns: 
    annotation = wav.replace(".wav", ".json")
    if not osp.isfile(annotation):
        print(f"pass {wav}")
        continue
    #데이터 처리 
    audio_arr, text_arr = data_process(wav, annotation)
    for i, audio in enumerate(audio_arr):
        audios.append(dict(
                path=osp.basename(wav),
                array=audio,
                sampling_rate = 16000,
            )) 
        texts.append(text_arr[i]) 

 number of files: 150
pass /home/yerang/문서/ASR/python_speech_recognition/aihub/06.지하철,버스/02.지하철안/06_08_008408_210918_SN.wav
pass /home/yerang/문서/ASR/python_speech_recognition/aihub/06.지하철,버스/02.지하철안/06_08_014091_210927_SN.wav
pass /home/yerang/문서/ASR/python_speech_recognition/aihub/06.지하철,버스/02.지하철안/06_07_023474_211011_SN.wav
pass /home/yerang/문서/ASR/python_speech_recognition/aihub/06.지하철,버스/02.지하철안/06_07_022260_211010_SN.wav
pass /home/yerang/문서/ASR/python_speech_recognition/aihub/06.지하철,버스/02.지하철안/06_08_008953_210916_SN.wav
pass /home/yerang/문서/ASR/python_speech_recognition/aihub/06.지하철,버스/02.지하철안/06_08_007971_210909_SN.wav
pass /home/yerang/문서/ASR/python_speech_recognition/aihub/06.지하철,버스/02.지하철안/06_08_014914_211001_SN.wav
pass /home/yerang/문서/ASR/python_speech_recognition/aihub/06.지하철,버스/02.지하철안/06_08_021071_211006_SN.wav
pass /home/yerang/문서/ASR/python_speech_recognition/aihub/06.지하철,버스/02.지하철안/06_08_023518_211011_SN.wav
pass /home/yerang/문서/ASR/python_speech_recognition/aihub/06.

In [36]:
assert len(audios)==len(texts)
print(len(audios))
print(len(texts))

1310
1310


### __허깅페이스 데이터셋에 업로드(optional)__

> __datasets 라이브러리를 사용하여 커스텀 데이터셋을 업로드할 경우 허깅페이스 계정과 인증 토큰(WRITE 권한)이 필요합니다. 토큰은 설정에서 발급받을 수 있습니다.__

In [38]:
from datasets import Dataset

audio_dataset = Dataset.from_dict(dict(
    audio = audios,
    text = texts
))

In [40]:
audio_dataset

Dataset({
    features: ['audio', 'text'],
    num_rows: 1310
})

> __push_to_hub 함수를 이용하여 업로드할 수 있습니다. 레포지토리가 존재하지 않는다면 생성한 후 업로드합니다. private=True일 경우 본인의 액세스 토큰을 가지고 있는 사람만 접근가능합니다.__

In [41]:
# Repository 생성 & upload
REPO_NAME = "zzrng76/aihub-noise-dataset-subway" #{사용자 이름}/{생성할 레포지토리 이름}
AUTH_TOKEN = ... #개인 계정 토큰 입력
audio_dataset.push_to_hub(REPO_NAME, token=AUTH_TOKEN, private=True,)

Uploading the dataset shards:   0%|          | 0/3 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/539 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/zzrng76/aihub-noise-dataset-subway/commit/7f274d7e5f5c883f7328d0538334115209bd79ea', commit_message='Upload dataset', commit_description='', oid='7f274d7e5f5c883f7328d0538334115209bd79ea', pr_url=None, pr_revision=None, pr_num=None)